*C++ presentation of 'Templet application' interface and model*

In [1]:
#include <string>
#include <list>
#include <map>

using namespace std;

struct TEMPLET_APP{

typedef std::string NAME;// to whom the token was issued
typedef long        TOKEN;

struct  SESSION{
    bool active = false;
    NAME name;      // who opened the session
    bool resource;  // what kind of access he(she) has
    bool user;
    bool admin;
};

struct ACCESS{
    TOKEN token;
    NAME  name;
    bool  resource;
    bool  user;
    bool  admin;
};

list<ACCESS>       access_table;
map<TOKEN,ACCESS*> access_by_token_indx;
map<NAME, ACCESS*> access_by_name_indx;    
    
bool open_session(/*in*/TOKEN t,/*out*/SESSION&s){
    try{
        ACCESS* acs = access_by_token_indx[t];
        s.active = true;
        s.name = acs->name; s.resource = acs->resource;
        s.user = acs->user; s.admin = acs->admin;
        return true;
    }
    catch(...){
        s.active = false;
        return false;
    }
}
    
bool close_session(/*in*/SESSION& s){
    if(s.active){s.active=false;return true;}
    else return false;
}

typedef long ORDINAL;
typedef long TAG;
typedef long ID;

struct  DATA{ long size; void* data; };

struct EVENT{
    ORDINAL  ord;
    TAG      tag;
    DATA     data;
    bool     is_query;
    NAME     writer_name;
};

list<EVENT>         event_table;
map<ORDINAL,EVENT*> event_by_ord_indx;    
    
bool write_event(/*in*/SESSION&s,/*in*/TAG t,/*in*/DATA d,/*out*/ORDINAL&ord){
    if(!(s.active && s.resource))return false;
    
    EVENT ev;
    
    ev.ord = event_table.size()+1; 
    ev.tag = t;
    ev.data = d;
    ev.is_query = false;
    ev.writer_name = s.name;
    
    event_table.push_back(ev);
    event_by_ord_indx[ev.ord] = &(event_table.back());
    
    return true;
}
                
bool read_event(/*in*/SESSION&s, /*in*/ORDINAL ord,/*out*/EVENT&ev){
    if(!(s.active && s.resource))return false;
    
    try{
        ev = *event_by_ord_indx[ord];
        return true;
    }
    catch(...){
        return false;
    }
}

struct ANSWER{
    ORDINAL ord;
    NAME    name;
    DATA    data;
};

list<ANSWER>         answer_table;
map<ORDINAL,ANSWER*> answer_by_ord_indx;
    
bool reply_on_query(/*in*/SESSION&s,/*in*/ORDINAL ord,/*in*/DATA d){
    if(!(s.active && s.resource))return false;
    
    try{
        ANSWER answ;
        
        answ.ord  = ord; 
        answ.name = event_by_ord_indx[ord]->writer_name;
        answ.data = d;
    
        answer_table.push_back(answ);
        answer_by_ord_indx[ord] = &(answer_table.back());
    
        return true;
    }
    catch(...){
        return false;
    }
}

bool write_query(/*in*/SESSION&s,/*in*/TAG t,/*in*/DATA d,/*out*/ORDINAL&){
    if(!(s.active && s.user))return false;
    
    EVENT ev;
    
    ev.ord = event_table.size()+1; 
    ev.tag = t;
    ev.data = d;
    ev.is_query = true;
    ev.writer_name = s.name;
    
    event_table.push_back(ev);
    event_by_ord_indx[ev.ord] = &(event_table.back());
    
    return true;
}
            
bool read_answer(/*in*/SESSION&s,/*in*/ORDINAL ord,/*out*/DATA&d){
    if(!(s.active && s.user))return false;
    
    try{
        ANSWER* answ = answer_by_ord_indx[ord];
        if(s.name != answ->name) return false;
        d = answ->data;
        return true;
    }
    catch(...){
        return false;
    }
}
////////////////////////////////////////////////////////////////
struct BLOB{
    ID   id;
    DATA data;
    NAME data_name;  
    NAME writer_name;
};

bool upload_blob(/*in*/SESSION&s,/*in*/DATA,/*in*/NAME,/*out*/ID&);
bool download_blob(/*in*/SESSION&s,/*in*/ID,/*out*/BLOB&);
bool delete_blob(/*in*/SESSION&s,/*in*/ID);
////////////////////////////////////////////////////////////////
};